####################### Optimised Model to Operate Sobol Simulations for PARASOL_LCA model #########################

THIS IS NOT A FINAL VERSION. THIS IS THE START OF AN OPTIMIZED MODEL THAT WAS INTERRUPTED TO FOCUS ON FINALIZING THE ORIGINAL PARASOL_LCA MODEL WHICH CONSTITUTES AND ESSENTIAL STEP BEFORE THE OPTIMIZATION. 
IT SHOULD BE NOTED THAT THE WORK SHOULD BE CONTINUED BY THE FUTURE CANDIDATE. THIS IS WHY I PREPARED A STEP-BY-STEP GUIDE ON WHICH WE RELIED TO DEVELOP THIS MODEL.

This Notebook represents the Optimised Version on which the future candidate will be working to run Sobol. Benoît GSCHWIND and Scarlett TANNOUS proposed and worked together to establish the methodology and choose the Brightway2's functions to elaborate this work. It is not completed. However, it is in progress which means that 
the work has started with the first components which can also serve as straightforward examples on which the candidate can rely to continue the work similarly. It is indispensable to accompany this Notebook with the Original PARASOL_LCA Parameterized model that Scarlett TANNOUS and Romain BESSEAU have developped.

# Loading Libraries and Databases

This section is similar to the first few lines of the original PARASOL_LCA model. It includes the loading of Libraries and Databases that will be used in the project.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw
from scipy.interpolate import InterpolatedUnivariateSpline
plt.style.use('ggplot')
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
from SALib.sample import saltelli
from SALib.analyze import sobol

ModuleNotFoundError: No module named 'SALib'

In [4]:
bw.projects.current

'default'

In [5]:
bw.projects.set_current('incer_acv')

In [ ]:
#bw.databases
#bw.bw2setup()
'''
fpei34 =  r'C:\Users\Scarlett Tannous\MPT_Python_code\Brightway_Training\Brightway&Python\ecoinvent 3.4_cutoff_ecoSpold02_COPYSOB\datasets'
if 'ecoinvent 3.4 cutoff' in bw.databases:
    print("Database has already been imported")
else:
    ei34 = bw.SingleOutputEcospold2Importer(fpei34, 'ecoinvent 3.4 cut off')
    ei34.apply_strategies()
    ei34.statistics()
'''
#ei34.write_database()

In [6]:
eidb = bw.Database('ecoinvent 3.4 cut off')
bio3 = bw.Database('biosphere3')

In [7]:
len(bio3)

0

In [8]:
len(eidb)

0

In [82]:
'''
if 'incer-acv' in [x[0] for x in bw.projects.report()]:
    bw.projects.delete_project('incer-acv')
    bw.projects.purge_deleted_directories
'''

"\nif 'incer-acv' in [x[0] for x in bw.projects.report()]:\n    bw.projects.delete_project('incer-acv')\n    bw.projects.purge_deleted_directories\n"

# Print Functions

This section includes different functions allowing to display datasets with different level of details. 

In [10]:
def print_data(act):
    df = pd.DataFrame(index = ['amount','unit','key','type'])
    for (i,exc) in enumerate(act.exchanges()):
        df[str(bw.get_activity(exc.input.key))] = [exc.amount, exc.unit, exc.input.key,exc['type']]
    return df.T

In [11]:
def print_data2(act):
    df = pd.DataFrame(index = ['amount','unit'])
    for (i,exc) in enumerate(act.exchanges()):
        df[str(bw.get_activity(exc.input.key))] = [exc.amount, exc.unit]
    return df.T

In [12]:
def print_data3(act):
    print(act)
    df = pd.DataFrame(index = ['amount','unit','key'])
    for exc in act.exchanges():
        if exc['name'] in df.columns:
            df[exc['name']].loc['amount'] += exc.amount
        else:
            df[exc['name']] = [exc.amount, exc.unit, exc.input.key]
    return df.T

# PV System

In this section the future candidate has to rely on Part_1 section of PARASOL_LCA model notebook to be able to convert it into the proposed functions. 

PARASOL_LCA model should be open parallelly with this notebook and the candidate should go through PARASOL_LCA function by function (or cell by cell) in order to do the following:
1- In PARASOL_LCA model, detect the functions that generate a change in the original dataset.
For example, the function 'generate_aluminium_alloy_dataset' in the original model is a function that creates a copy of the original aluminum dataset in order to integrate to it additional modification.
2- In PARASOL_LCA model, identify the parameters on which each generating function relies on.
For example, the function 'generate_aluminium_alloy_dataset' in the original model has 2 parameters (or input variables) : 'share_recycled_aluminium' and 'elec_dataset'.
3- The identified input variables should be integrated in the datasetX, especifically in the concerned exchangeX itself using the 'exchangeX['formula']'. It is where the parameters should be integrated with the appropriate mathematical relationship. The person should be careful to revise the original code and understand each line of it to be able to detect each parameter to which dataset it is related.
For example, since the parameter 'share_recycled_aluminium' is connected the following 2 exchanges: exc = 'aluminium, cast alloy' and exc = 'aluminium scrap, new, Recycled Content cut-off', the candidate should stock in the exc['formula'] the corresponding values with their approriate relationship with the parameter as follows:  exc['formula'] = '%f * (1.0 - share_recycled_aluminium)'%(aluminium_amount,).
4- In the optimised version, for each component, at the end of the corresponding section, I compared the original dataset to the adjusted one to be sure that the integrated change is well modeled.
This can serve also the candidate to be reassured of his/her steps.
5- After finishing the transformation of each generating function in the original PARASOL_LCA model into a parameterized function as proposed in the steps 1-4, the candidate will have to stock the parameters in a list called my_parameters = [] as represented in the Notebook 'Parameters_for_Optimised_Model_Sobol_DRAFT1_TANNOUS_Scarlett' . I separated the set of paramters in a separate Notebook just to be visually and technically more practical to have 2 Notebooks on 2 different screens. However, the candidate can stick to 1 Notebook and put them all together in a section. 
In this parmeters' list, the candidate will have to list each parameter and attribute it to the corresponding database, code, name, and amount. 
For example, for the alu_adjusted, the following information will be stocked in the my_paramters[] list:

{'database': alu_adjusted['database'],
 'code': alu_adjusted['code'],
 'name': 'share_recycled_aluminium',
 'amount': 0.0}
 
6- After stocking all the parameters in the list of my_parameter[] in the Notebook 'Parameters_for_Optimised_Model_Sobol_DRAFT1_TANNOUS_Scarlett', the candidate will have to follow the proposed lines of codes in this notebook and try to adjust them according to the added parameters.

7- After finishing the 'parameters' section of 'Parameters_for_Optimised_Model_Sobol_DRAFT1_TANNOUS_Scarlett', the candidate will have to start the execution of the scenarios. At this level, personal effort and researched should be done to understand what is the purpose of each function. However, in order to help at this level, I highly recommend to check the following reference https://salib.readthedocs.io/en/latest/ and go into the details of the Sobol code that Romain Sacchi has extracted and organised in a Notebook called 'Sobol'. At this level, I cannot offer more support as I stopped my investigation here. It is where additional work and research should be done to continue modeling and exploring the functions and the methods available in this library.

## Electricity

In [13]:
elec_dataset = [act for act in eidb if 'market group for electricity, medium voltage' == act['name'] and 'ENTSO-E' == act['location']][0]
elec_dataset

'market group for electricity, medium voltage' (kilowatt hour, ENTSO-E, None)

In [14]:
electricity_activity_w_location = [
        {'name': 'market group for electricity, medium voltage', 'location': 'ENTSO-E', 'varname': 'eu_electricity_mix'},
        {'name': 'electricity, high voltage, production mix', 'location': 'CN-GD', 'varname': 'cn_electricity_mix'},
        {'name': 'electricity, high voltage, production mix', 'location': 'FR', 'varname': 'fr_electricity_mix'},
        {'name': 'market for electricity, high voltage', 'location': 'US-WECC', 'varname': 'us_electricity_mix'},
        {'name': 'market for electricity, high voltage', 'location': 'RoW', 'varname': 'row_electricity_mix'}
]

for x in electricity_activity_w_location:
    elec_dataset = [act for act in eidb if x['name'] == act['name'] and x['location'] == act['location']]
    if len(elec_dataset) != 1:
        print("ERROR", x['name'], x['location'])
    x['activity'] = elec_dataset[0]
               
print(electricity_activity_w_location)

[{'name': 'market group for electricity, medium voltage', 'location': 'ENTSO-E', 'varname': 'eu_electricity_mix', 'activity': 'market group for electricity, medium voltage' (kilowatt hour, ENTSO-E, None)}, {'name': 'electricity, high voltage, production mix', 'location': 'CN-GD', 'varname': 'cn_electricity_mix', 'activity': 'electricity, high voltage, production mix' (kilowatt hour, CN-GD, None)}, {'name': 'electricity, high voltage, production mix', 'location': 'FR', 'varname': 'fr_electricity_mix', 'activity': 'electricity, high voltage, production mix' (kilowatt hour, FR, None)}, {'name': 'market for electricity, high voltage', 'location': 'US-WECC', 'varname': 'us_electricity_mix', 'activity': 'market for electricity, high voltage' (kilowatt hour, US-WECC, None)}, {'name': 'market for electricity, high voltage', 'location': 'RoW', 'varname': 'row_electricity_mix', 'activity': 'market for electricity, high voltage' (kilowatt hour, RoW, None)}]


## Aluminium

### Original dataset

In [15]:
alu = [act for act in eidb if 'aluminium alloy production, AlMg3' == act['name']
       and 'RER' in act['location']][0]
alu

'aluminium alloy production, AlMg3' (kilogram, RER, None)

### Parameterized function

In [15]:
def ensure_aluminium_adjusted():
    alu = [act for act in eidb if 'aluminium alloy production, AlMg3' in act['name']and 'RER' in act['location']][0]
    alu_adjusted = [act for act in eidb if 'aluminium alloy production, AlMg3 adjusted' in act['name'] 
                    and 'RoW' in act['location']]
    # Amount of aluminium considered
    for exc in alu.exchanges():
        if exc['name'] == 'aluminium, cast alloy':
            aluminium_amount = exc.amount
    
    if len(alu_adjusted) == 0:
        alu = [act for act in eidb if 'aluminium alloy production, AlMg3' in act['name']and 'RER' in act['location']][0]
        alu_adjusted = alu.copy()
        alu_adjusted['name'] = 'aluminium alloy production, AlMg3 adjusted'
        alu_adjusted['authors'] = 'Romain Besseau'
        alu_adjusted['comment'] = 'Dataset adjusted with the share of secondary aluminium used and the electricity mix used'
        
        #Adding secondary aluminium
        aluminium_scrap = [act for act in eidb if 'aluminium scrap, new, Recycled Content cut-off' in act['name']][0]

        new_exc = alu_adjusted.new_exchange(input = aluminium_scrap.key, name = aluminium_scrap['name'], 
                                        amount = 0, unit = 'kg', categories = '', type = 'technosphere')
        new_exc.save()
        alu_adjusted.save()
        
    for exc in alu_adjusted.exchanges():
        if exc['name'] == 'aluminium, cast alloy':
            exc['formula'] = '%f * (1.0 - share_recycled_aluminium)'%(aluminium_amount,)
            exc.save()
        if exc['name'] == 'aluminium scrap, new, Recycled Content cut-off':
            exc['formula'] = '%f * (share_recycled_aluminium)'%(aluminium_amount,)
            exc.save()
        
        if exc['name'] == 'electricity, high voltage, aluminium industry':
            electricity_amount = exc
            exc['formula'] = '%f * default_electricity_mix'%(electricity_amount['amount'])
            exc.save()
        
    
    for x in electricity_activity_w_location:
        act = x['activity']
        exc = alu_adjusted.new_exchange(
            input = act.key,
            name="%s %s"%(x['name'], x['location']),
            amount=0.0,
            unit=electricity_amount['unit'],
            category='',
            type=electricity_amount['type'],
            formula='%f * %s'%(electricity_amount['amount'], x['varname']))
        exc.save()
    alu_adjusted.save()    
    
    return alu_adjusted

In [16]:
alu_adjusted = ensure_aluminium_adjusted()
alu_adjusted

'aluminium alloy production, AlMg3 adjusted' (kilogram, RER, None)

In [16]:
alu_adjusted = [act for act in eidb if 'aluminium alloy production, AlMg3 adjusted' == act['name']][0]
print_data(alu_adjusted)

,amount,unit,key,type
"'market for electricity, high voltage, aluminium industry' (kilowatt hour, IAI Area, EU27 & EFTA, None)",1.59,kilowatt hour,"(ecoinvent 3.4 cut off, e45c6da9a7c4e6b5d4bef9...",technosphere
"'market for manganese' (kilogram, GLO, None)",0.00508,kilogram,"(ecoinvent 3.4 cut off, a5ac0e818e17156882d192...",technosphere
"'aluminium alloy production, AlMg3 adjusted' (kilogram, RER, None)",1,kilogram,"(ecoinvent 3.4 cut off, f0cbf763bfc944e2b9906b...",production
"'market for magnesium' (kilogram, GLO, None)",0.0305,kilogram,"(ecoinvent 3.4 cut off, cab8a25e39b619b55b0d11...",technosphere
"'market for silicon, metallurgical grade' (kilogram, GLO, None)",0.00406,kilogram,"(ecoinvent 3.4 cut off, 3188d23f89079fe1ac63e4...",technosphere
"'market for copper' (kilogram, GLO, None)",0.00102,kilogram,"(ecoinvent 3.4 cut off, d6f0a3a0933866c2f8d7eb...",technosphere
"'market for aluminium, cast alloy' (kilogram, GLO, None)",0.965,kilogram,"(ecoinvent 3.4 cut off, bb3477bde0c4f3663411a7...",technosphere
"'market for zinc' (kilogram, GLO, None)",0.00203,kilogram,"(ecoinvent 3.4 cut off, 002ebeb1ca3506ef21cf04...",technosphere
"'market for cast iron' (kilogram, GLO, None)",0.00406,kilogram,"(ecoinvent 3.4 cut off, 0e718dd998cd3c6ca2aac0...",technosphere
"'market for chromium' (kilogram, GLO, None)",0.00305,kilogram,"(ecoinvent 3.4 cut off, 3ecd7cb07efe822b692dd3...",technosphere


### Comparision of original and adjusted ds

In [21]:
alu_adjusted = [act for act in eidb if 'aluminium alloy production, AlMg3 adjusted' in act['name']][0]
alu_adjusted

'aluminium alloy production, AlMg3 adjusted' (kilogram, RER, None)

In [22]:
pd.concat([print_data2(alu),print_data2(alu_adjusted)], axis=1)

C:\Users\Scarlett Tannous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,unit,amount,unit
"'aluminium alloy production, AlMg3 adjusted' (kilogram, RER, None)",NaN,NaN,1,kilogram
"'aluminium alloy production, AlMg3' (kilogram, RER, None)",1,kilogram,NaN,NaN
"'aluminium scrap, new, Recycled Content cut-off' (kilogram, GLO, None)",NaN,NaN,0,kilogram
"'electricity, high voltage, production mix' (kilowatt hour, CN-GD, None)",NaN,NaN,0,kilowatt hour
"'electricity, high voltage, production mix' (kilowatt hour, FR, None)",NaN,NaN,0,kilowatt hour
"'market for aluminium, cast alloy' (kilogram, GLO, None)",0.965,kilogram,0.965,kilogram
"'market for cast iron' (kilogram, GLO, None)",0.00406,kilogram,0.00406,kilogram
"'market for chromium' (kilogram, GLO, None)",0.00305,kilogram,0.00305,kilogram
"'market for copper' (kilogram, GLO, None)",0.00102,kilogram,0.00102,kilogram
"'market for electricity, high voltage' (kilowatt hour, RoW, None)",NaN,NaN,0,kilowatt hour


## Mounting system

### Roof

#### Original roof ds

In [17]:
roof_system = [act for act in eidb if
               'photovoltaic mounting system production, for slanted-roof installation' == act['name'] 
                and 'RER' in act['location']][0]
roof_system

'photovoltaic mounting system production, for slanted-roof installation' (square meter, RER, None)

#### Modifying function

In [19]:
def ensure_roof_mounting_system():
    
    roof_system = [act for act in eidb if
                       'photovoltaic mounting system production, for slanted-roof installation' == act['name'] 
                        and 'RER' in act['location']][0]
    
    for exc in roof_system.exchanges():
            if exc['name'] == 'aluminium, wrought alloy':
                aluminium_amount = exc.amount
                
    roof_system_with_recycling = [act for act in eidb if
      'photovoltaic mounting system production, for slanted-roof installation with recycling' == act['name']]
    
    
    if len(roof_system_with_recycling) == 0:
        roof_system = [act for act in eidb if
                       'photovoltaic mounting system production, for slanted-roof installation' == act['name'] 
                        and 'RER' in act['location']][0]
        roof_system_with_recycling = roof_system.copy()
        roof_system_with_recycling['name'] = 'photovoltaic mounting system production, for slanted-roof installation with recycling'
        roof_system_with_recycling['authors'] = 'Romain Besseau'
        roof_system_with_recycling['comment'] = 'Copy of photovoltaic mounting system production, for slanted-roof installation with recycling activites added (scrap steel and scrap aluminium)'
        roof_system_with_recycling.save()
        scrap_aluminium = [act for act in eidb if 'market for scrap aluminium' in act['name']
                                   and 'Europe' in act['location']][0]  
        
        scrap_steel = [act for act in eidb if 'market for scrap steel' 
               in act['name'] and 'Europe' in act['location']][0]

        for exc in roof_system_with_recycling.exchanges():
            if exc['name'] == 'aluminium, wrought alloy':
                aluminium_amount = exc.amount
                new_exc = roof_system_with_recycling.new_exchange(input = scrap_aluminium.key, 
                                                                  name = scrap_aluminium['name'], 
                                                                  amount =  - aluminium_amount,
                                                                  unit = scrap_aluminium['unit'],
                                                                  type = 'technosphere')
                new_exc.save()

            if exc['name'] == 'steel, low-alloyed, hot rolled':
                steel_amout = exc.amount
                new_exc = roof_system_with_recycling.new_exchange(input = scrap_steel.key,
                                                                  name = scrap_steel['name'],
                                                                  amount =  - steel_amout, 
                                                                  unit = scrap_steel['unit'], 
                                                                  type = 'technosphere')
                new_exc.save()


        #add the alu_adjusted with 0 amount
        alu_adjusted = [act for act in eidb if 'aluminium alloy production, AlMg3 adjusted' == act['name']][0]
        new_exc = roof_system_with_recycling.new_exchange(input = alu_adjusted.key,
                                                                  name = alu_adjusted['name'],
                                                                  amount =  0, 
                                                                  unit = alu_adjusted['unit'], 
                                                                  type = 'technosphere')
        new_exc.save()
        roof_system_with_recycling.save()
        
    roof_system_with_recycling = [act for act 
                                  in eidb if
                                  'photovoltaic mounting system production, for slanted-roof installation with recycling'
                                  in act['name']][0]

    for exc in roof_system_with_recycling.exchanges():
        if exc['name'] == 'aluminium, wrought alloy':
            exc['amount'] = 0.0
            exc.save()
        
        if exc['name'] == 'aluminium alloy production, AlMg3 adjusted':
            exc['amount'] = aluminium_amount
            exc.save()
            
    roof_system_with_recycling.save() 
    return roof_system_with_recycling

In [20]:
roof_system_with_recycling = ensure_roof_mounting_system()

In [21]:
roof_system_with_recycling

'photovoltaic mounting system production, for slanted-roof installation with recycling' (square meter, RER, None)

#### Comparision original and modified roof ds

In [22]:
pd.concat([print_data2(roof_system),print_data2(roof_system_with_recycling)], axis = 1)

C:\Users\Scarlett Tannous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,unit,amount,unit
"'aluminium alloy production, AlMg3 adjusted' (kilogram, RER, None)",NaN,NaN,2.8355,kilogram
"'market for aluminium, wrought alloy' (kilogram, GLO, None)",2.8355,kilogram,0,kilogram
"'market for corrugated board box' (kilogram, GLO, None)",0.13332,kilogram,0.13332,kilogram
"'market for polyethylene, high density, granulate' (kilogram, GLO, None)",0.0014033,kilogram,0.0014033,kilogram
"'market for polystyrene, high impact' (kilogram, GLO, None)",0.0070167,kilogram,0.0070167,kilogram
"'market for scrap aluminium' (kilogram, Europe without Switzerland, None)",NaN,NaN,-2.8355,kilogram
"'market for scrap steel' (kilogram, Europe without Switzerland, None)",NaN,NaN,-1.4999,kilogram
"'market for section bar extrusion, aluminium' (kilogram, GLO, None)",3.0312,kilogram,3.0312,kilogram
"'market for sheet rolling, steel' (kilogram, GLO, None)",1.4999,kilogram,1.4999,kilogram
"'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None)",1.4999,kilogram,1.4999,kilogram


### Ground

#### Original ground ds

In [23]:
ground_system = [act for act in eidb if 'photovoltaic mounting system production, for 570kWp open ground module' in act['name']][0]
ground_system

'photovoltaic mounting system production, for 570kWp open ground module' (square meter, GLO, None)

In [24]:
print_data(ground_system)

,amount,unit,key,type
"'market for aluminium, wrought alloy' (kilogram, GLO, None)",3.98,kilogram,"(ecoinvent 3.4 cut off, d7b2398f8a10f8a4f80978...",technosphere
"'market for polystyrene, high impact' (kilogram, GLO, None)",0.0045455,kilogram,"(ecoinvent 3.4 cut off, 6b34f1cda0bf360140a979...",technosphere
"'market for reinforcing steel' (kilogram, GLO, None)",7.25,kilogram,"(ecoinvent 3.4 cut off, bb415d7f49afbe3c0fe672...",technosphere
"'market for zinc coat, coils' (square meter, GLO, None)",0.11,square meter,"(ecoinvent 3.4 cut off, 18b324dfe0b471b64650bc...",technosphere
"'market for steel, chromium steel 18/8, hot rolled' (kilogram, GLO, None)",0.25,kilogram,"(ecoinvent 3.4 cut off, dd6d2a4b0531083b2aec3f...",technosphere
"'photovoltaic mounting system production, for 570kWp open ground module' (square meter, GLO, None)",1,square meter,"(ecoinvent 3.4 cut off, 08d8b611f677c72c5ceafe...",production
"'market for polyethylene, high density, granulate' (kilogram, GLO, None)",0.00090909,kilogram,"(ecoinvent 3.4 cut off, c6d5b7ed36ef2c2d392473...",technosphere
"'market for section bar rolling, steel' (kilogram, GLO, None)",6.15,kilogram,"(ecoinvent 3.4 cut off, 10a99599dfc3a449f573b8...",technosphere
"'market for corrugated board box' (kilogram, GLO, None)",0.086364,kilogram,"(ecoinvent 3.4 cut off, 04d83c7892991a382a8177...",technosphere
"'market for wire drawing, steel' (kilogram, GLO, None)",1.1,kilogram,"(ecoinvent 3.4 cut off, baf1a7b9c00f8093f933e5...",technosphere


#### Modifying ground function

In [25]:
ground_system = [act for act in eidb if 
                     'photovoltaic mounting system production, for 570kWp open ground module'
                     in act['name']][0]

In [27]:
def ensure_ground_mounting_system():
    ground_system = [act for act in eidb if 
                     'photovoltaic mounting system production, for 570kWp open ground module'
                     in act['name']][0]

    ground_system_adjusted = [act for act in eidb 
                              if 'photovoltaic mounting system production, for 570kWp open ground module adjusted' == act['name']]
    
    for exc in ground_system.exchanges():
        if exc['name'] == 'aluminium, wrought alloy':
            aluminium_amount = exc['amount']
    
    if len(ground_system_adjusted) == 0: 
        ground_system = [act for act in eidb if 
                     'photovoltaic mounting system production, for 570kWp open ground module'
                     in act['name']][0]
        ground_system_adjusted = ground_system.copy()
        ground_system_adjusted['name'] = 'photovoltaic mounting system production, for 570kWp open ground module adjusted'
        ground_system_adjusted['authors'] = 'Romain Besseau'
        ground_system_adjusted['comment'] = 'Dataset adjusted with the share of secondary aluminium used and the recycling rate'
        ground_system_adjusted.save()
        #Changing the aluminium dataset to the adjusted one.
        alu_adjusted = [act for act in
                eidb if 'aluminium alloy production, AlMg3 adjusted' == act['name']][0]
        
        new_exc = ground_system_adjusted.new_exchange(input = alu_adjusted.key,
                                                      name = alu_adjusted['name'],
                                                      amount =  0, 
                                                      unit = alu_adjusted['unit'], 
                                                      type = 'technosphere')
        new_exc.save()
        ground_system_adjusted.save()       
        
        for exc in ground_system_adjusted.exchanges():
            if exc['name'] == 'aluminium, wrought alloy':
                exc['amount'] = 0
                exc.save()      
                
            if exc['name'] == 'aluminium alloy production, AlMg3 adjusted':
                exc['amount'] = aluminium_amount
                exc.save()           

        ground_system_adjusted.save() 
        
    else:
        ground_system_adjusted = [act for act in eidb 
          if 'photovoltaic mounting system production, for 570kWp open ground module adjusted' == act['name']][0]
        
    return ground_system_adjusted

In [28]:
ground_system_adjusted = ensure_ground_mounting_system()

In [29]:
print_data2(ground_system_adjusted)

,amount,unit
"'market for aluminium, wrought alloy' (kilogram, GLO, None)",0,kilogram
"'market for polystyrene, high impact' (kilogram, GLO, None)",0.0045455,kilogram
"'market for reinforcing steel' (kilogram, GLO, None)",7.25,kilogram
"'market for zinc coat, coils' (square meter, GLO, None)",0.11,square meter
"'market for steel, chromium steel 18/8, hot rolled' (kilogram, GLO, None)",0.25,kilogram
"'photovoltaic mounting system production, for 570kWp open ground module adjusted' (square meter, GLO, None)",1,square meter
"'market for polyethylene, high density, granulate' (kilogram, GLO, None)",0.00090909,kilogram
"'market for section bar rolling, steel' (kilogram, GLO, None)",6.15,kilogram
"'market for corrugated board box' (kilogram, GLO, None)",0.086364,kilogram
"'market for wire drawing, steel' (kilogram, GLO, None)",1.1,kilogram


#### Comparision original and modified ground ds

In [49]:
pd.concat([print_data2(ground_system),print_data2(ground_system_adjusted)], axis = 1)

C:\Users\Scarlett Tannous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,unit,amount,unit
"'Occupation, industrial area' (square meter-year, None, ('natural resource', 'land'))",141,square meter-year,141,square meter-year
"'Transformation, from pasture, man made' (square meter, None, ('natural resource', 'land'))",4.7,square meter,4.7,square meter
"'Transformation, to industrial area' (square meter, None, ('natural resource', 'land'))",4.7,square meter,4.7,square meter
"'aluminium alloy production, AlMg3 adjusted' (kilogram, RER, None)",NaN,NaN,3.98,kilogram
"'market for aluminium, wrought alloy' (kilogram, GLO, None)",3.98,kilogram,0,kilogram
"'market for concrete, normal' (cubic meter, CH, None)",1.04537e-06,cubic meter,1.04537e-06,cubic meter
"'market for concrete, normal' (cubic meter, RoW, None)",0.000540621,cubic meter,0.000540621,cubic meter
"'market for corrugated board box' (kilogram, GLO, None)",0.086364,kilogram,0.086364,kilogram
"'market for polyethylene, high density, granulate' (kilogram, GLO, None)",0.00090909,kilogram,0.00090909,kilogram
"'market for polystyrene, high impact' (kilogram, GLO, None)",0.0045455,kilogram,0.0045455,kilogram


In [ ]:
add the 2 of them:
act['fomrula'] = %f roof (this is o or 1) %(amount)

## Electrical Installation

In [30]:
electrical_installation_3kW = [act for act in eidb if 'photovoltaics, electric installation for 3kWp module, at building' == act['name'] and 'RoW' in act['location']][0]
electrical_installation_3kW

'photovoltaics, electric installation for 3kWp module, at building' (unit, RoW, None)

In [31]:
unit_of_equivalent_3kW_electrical_installation = InterpolatedUnivariateSpline([3, 570],
                                                                              [33/33, 
                                                                               1570/33],
                                                                              k=1)
unit_of_equivalent_3kW_electrical_installation(10)

array(1.57500935)

## Inverter

In [ ]:
(P = 10, elec_dataset = None, recycling_rate = 0)

### Inverter 2.5kW

In [32]:
alkyl_paint = [act for act in eidb if 'alkyd paint, white, without solvent, in 60% solution state' in act['name']][0]
glass_fiber = [act for act in eidb if 'glass fibre reinforced plastic, polyamide, injection moulded' in act['name']][0]
glass_plastic = [act for act in eidb if 'glass fibre reinforced plastic, polyester resin, hand lay-up' in act['name']][0]
injection_moulding = [act for act in eidb if 'injection moulding' in act['name'] and 'GLO' in act['location']][0]
lubricating_oil = [act for act in eidb if 'lubricating oil' in act['name'] and 'GLO' in act['location']][0]
polyethylene = [act for act in eidb if 'polyethylene, high density, granulate' in act['name'] and 'GLO' in act['location']][0]
waste_mineral = [act for act in eidb if 'market for waste mineral oil' in act['name'] and 'Europe' in act['location']][0]
waste_plastic = [act for act in eidb if 'market for waste plastic, industrial electronics' in act['name']][0]

In [33]:
act = [act for act in eidb if 'alkyd paint, white, without solvent, in 60% solution state' in act['name']][0]
act = [act for act in eidb if 'glass fibre reinforced plastic, polyamide, injection moulded' in act['name']][0]
act = [act for act in eidb if 'glass fibre reinforced plastic, polyester resin, hand lay-up' in act['name']][0]
act = [act for act in eidb if 'injection moulding' in act['name'] and 'GLO' in act['location']][0]
act = [act for act in eidb if 'lubricating oil' in act['name'] and 'GLO' in act['location']][0]
act = [act for act in eidb if 'polyethylene, high density, granulate' in act['name'] and 'GLO' in act['location']][0]
act = [act for act in eidb if 'market for waste mineral oil' in act['name'] and 'Europe' in act['location']][0]
act = [act for act in eidb if 'market for waste plastic, industrial electronics' in act['name']][0]

In [34]:
def ensure_inverter_2_5kW():
    inverter_2_5kW = [act for act in eidb if 'inverter production, 2.5kW' in act['name'] and 'RER' in act['location']][0]
    inverter_2_5kW_adjusted = [act for act in eidb if 'inverter production, 2.5kW adjusted' in act['name'] and 'RER' in act['location']]
    

    if len(inverter_2_5kW_adjusted) == 0:
        inverter_2_5kW_adjusted = inverter_2_5kW.copy()
        inverter_2_5kW_adjusted['name'] = 'inverter production, 2.5kW adjusted'
        inverter_2_5kW_adjusted['location'] = 'RER' 
        inverter_2_5kW_adjusted['author'] = 'Scarlett Tannous'
        inverter_2_5kW_adjusted['comment'] = 'the exchanges missing from inverter production, 2.5kW and presend in inverter production, 500kW were added' 
        inverter_2_5kW_adjusted.save()
        
        #activities present in inverter 500kW but not in 2.5kW
        act = [act for act in eidb if 'alkyd paint, white, without solvent, in 60% solution state' in act['name']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'glass fibre reinforced plastic, polyamide, injection moulded' in act['name']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'glass fibre reinforced plastic, polyester resin, hand lay-up' in act['name']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'injection moulding' in act['name'] and 'GLO' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'lubricating oil' in act['name'] and 'GLO' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'polyethylene, high density, granulate' in act['name'] and 'GLO' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'market for waste mineral oil' in act['name'] and 'Europe' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        act = [act for act in eidb if 'market for waste plastic, industrial electronics' in act['name']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
                   
        #adding the recycling
        #scrap aluminium
        act = [act for act in eidb if 'market for scrap alu' in act['name'] and 'Europe' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        #scrap steel
        act = [act for act in eidb if 'market for scrap steel' in act['name'] and 'Europe' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        #scrap copper
        act = [act for act in eidb if 'market for scrap copper' in act['name'] and 'Europe' in act['location']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
       
        #scrap electronic
        act = [act for act in eidb if 'market for electronics scrap' in act['name']][0]
        new_exc = inverter_2_5kW_adjusted.new_exchange(input = act.key, name = act['name'], amount = 0, unit = act['unit'], type = 'technosphere')
        new_exc.save()
        
        inverter_2_5kW_adjusted.save()
        
        #electricity mix
        for exc in inverter_2_5kW_adjusted.exchanges():
            if exc['name'] == 'electricity, medium voltage':
                electricity_amount = exc
                exc['formula'] = '%f * default_electricity_mix'%(electricity_amount['amount'])
                exc.save()
          
        for x in electricity_activity_w_location:
            act = x['activity']
            exc = inverter_2_5kW_adjusted.new_exchange(
                input = act.key,
                name="%s %s"%(x['name'], x['location']),
                amount=0.0,
                unit=electricity_amount['unit'],
                category='',
                type=electricity_amount['type'],
                formula='%f * %s'%(electricity_amount['amount'], x['varname']))
            exc.save()
        inverter_2_5kW_adjusted.save()
        

        #recycling activities
        m_aluminium = 0
        m_copper = 0
        m_steel = 0
        m_electronic = 0
        #recycling_rate = 0

        for exc in inverter_2_5kW_adjusted.exchanges():
            if exc['name'] == 'aluminium, cast alloy':
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_aluminium += exc['amount']
            if exc['name'] == 'copper':
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_copper += exc['amount']
            if exc['name'] == 'steel, low-alloyed, hot rolled':
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_steel += exc['amount']
            if ('capacitor' in exc['name']) or ('diode' in exc['name']) or ('integrated circuit' in exc['name']) or ('printed wiring' in exc['name']) or ('resistor' in exc['name']) or ('transistor' in exc['name']):
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_electronic += exc['amount']
            exc.save()
        
        inverter_2_5kW_adjusted.save()

    else:
        inverter_2_5kW_adjusted = [act for act in eidb if 'inverter production, 2.5kW adjusted' in act['name']][0]
    
    return inverter_2_5kW_adjusted

In [240]:
inverter_2_5kW = [act for act in eidb if 'inverter production, 2.5kW' in act['name'] and 'RER' in act['location']][0]
inverter_2_5kW

'inverter production, 2.5kW' (unit, RER, None)

In [35]:
inverter_2_5kW_adjusted = [act for act in eidb if 'inverter production, 2.5kW adjusted' in act['name']][0]
inverter_2_5kW_adjusted

'inverter production, 2.5kW adjusted' (unit, RER, None)

In [65]:
inverter_2_5kW_adjusted = ensure_inverter_2_5kW()
inverter_2_5kW_adjusted 

'inverter production, 2.5kW adjusted' (unit, RER, None)

In [36]:
print_data2(inverter_2_5kW_adjusted)

,amount,unit
"'market for injection moulding' (kilogram, GLO, None)",71,kilogram
"'market for inductor, ring core choke type' (kilogram, GLO, None)",0.351,kilogram
"'market for waste plastic, industrial electronics' (kilogram, GLO, None)",-230,kilogram
"'market for transistor, wired, small size, through-hole mounting' (kilogram, GLO, None)",0.038,kilogram
"'market for used printed wiring boards' (kilogram, GLO, None)",-48.866,kilogram
"'market for printed wiring board, for through-hole mounting, Pb free surface' (square meter, GLO, None)",0.1123,square meter
"'market for lubricating oil' (kilogram, GLO, None)",881,kilogram
"'market for copper' (kilogram, GLO, None)",335,kilogram
"'market for polystyrene foam slab' (kilogram, GLO, None)",1.6,kilogram
"'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None)",1438,kilogram


the 2.5kW weighs 18.5 kg and the 500kW weighs 3000 kg.

### Inverter 500kW

In [57]:
PVC = [act for act in eidb if 'polyvinylchloride, suspension polymerised' in act['name'] and 'GLO' in act['location']][0]     
SAC = [act for act in eidb if 'styrene-acrylonitrile copolymer' in act['name'] and 'GLO' in act['location']][0]

In [66]:
def ensure_inverter_500kW():
    inverter_500kW = [act for act in eidb if 'inverter production, 500kW' in act['name'] and not 'car' in act['name'] and 'RER' in act['location']][0]
    inverter_500kW_adjusted = [act for act in eidb if 'inverter production, 500kW adjusted' == act['name']]
    
    if len(inverter_500kW_adjusted) == 0:
        inverter_500kW_adjusted = inverter_500kW.copy()
        inverter_500kW_adjusted['name'] = 'inverter production, 500kW adjusted'
        inverter_500kW_adjusted['location'] = 'RER' 
        inverter_500kW_adjusted['author'] = 'Scarlett Tannous'
        inverter_500kW_adjusted['comment'] = 'the exchanges missing from inverter production, 500kW and presend in inverter production, 500kW were added' 
        inverter_500kW_adjusted.save()
        
        #activities present in inverter 500kW but not in 2.5kW
        act = [act for act in eidb 
               if 'polyvinylchloride, suspension polymerised' in act['name'] and 'GLO' in act['location']][0]      
        new_exc = inverter_500kW_adjusted.new_exchange(input = act.key, 
                                                       name = act['name'], 
                                                       amount = 0, unit = act['unit'],
                                                       type = 'technosphere')
        new_exc.save()
          
        act = [act for act in eidb
               if 'styrene-acrylonitrile copolymer' in act['name'] and 'GLO' in act['location']][0]
        new_exc = inverter_500kW_adjusted.new_exchange(input = act.key, 
                                                       name = act['name'], 
                                                       amount = 0, unit = act['unit'],
                                                       type = 'technosphere')
        new_exc.save()
        inverter_500kW_adjusted.save()
        
        
        #electricity mix
        for exc in inverter_500kW_adjusted.exchanges():
            if exc['name'] == 'electricity, medium voltage':
                electricity_amount = exc
                exc['formula'] = '%f * default_electricity_mix'%(electricity_amount['amount'])
                exc.save()
          
        for x in electricity_activity_w_location:
            act = x['activity']
            exc = inverter_500kW_adjusted.new_exchange(
                input = act.key,
                name="%s %s"%(x['name'], x['location']),
                amount=0.0,
                unit=electricity_amount['unit'],
                category='',
                type=electricity_amount['type'],
                formula='%f * %s'%(electricity_amount['amount'], x['varname']))
            exc.save()
        inverter_500kW_adjusted.save()
        

        #recycling activities
        m_aluminium = 0
        m_copper = 0
        m_steel = 0
        m_electronic = 0
        recycling_rate = 0

        for exc in inverter_500kW_adjusted.exchanges():
            if exc['name'] == 'aluminium, cast alloy':
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_aluminium += exc['amount']
            if exc['name'] == 'copper':
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_copper += exc['amount']
            if exc['name'] == 'steel, low-alloyed, hot rolled':
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_steel += exc['amount']
            if ('capacitor' in exc['name']) or ('diode' in exc['name']) or ('integrated circuit' in exc['name']) or ('printed wiring' in exc['name']) or ('resistor' in exc['name']) or ('transistor' in exc['name']):
                exc['formula'] = '%f*%s'%(exc['amount'],recycling_rate)
                m_electronic += exc['amount']
            exc.save()
        
        inverter_500kW_adjusted.save()
        
    else:
        inverter_500kW_adjusted = [act for act in eidb if 'inverter production, 500kW adjusted' == act['name'] and 'RER' in act['location']][0]
     
    return inverter_500kW_adjusted

In [67]:
inverter_500kW_adjusted = ensure_inverter_500kW()

In [37]:
inverter_500kW_adjusted = [act for act in eidb if 'inverter production, 500kW adjusted' in act['name']][0]
print_data(inverter_500kW_adjusted)

,amount,unit,key,type
"'market for injection moulding' (kilogram, GLO, None)",71,kilogram,"(ecoinvent 3.4 cut off, 3781de39d137830e00d0d4...",technosphere
"'market for inductor, ring core choke type' (kilogram, GLO, None)",0.351,kilogram,"(ecoinvent 3.4 cut off, 4fe816c92b00c50236a58b...",technosphere
"'market for waste plastic, industrial electronics' (kilogram, GLO, None)",-230,kilogram,"(ecoinvent 3.4 cut off, 205d8c93fd695eca8f07d7...",technosphere
"'market for transistor, wired, small size, through-hole mounting' (kilogram, GLO, None)",0.038,kilogram,"(ecoinvent 3.4 cut off, 582fa68b18f99344993468...",technosphere
"'market for used printed wiring boards' (kilogram, GLO, None)",-48.866,kilogram,"(ecoinvent 3.4 cut off, 02a397f992ace13c52e2d5...",technosphere
"'market for printed wiring board, for through-hole mounting, Pb free surface' (square meter, GLO, None)",0.1123,square meter,"(ecoinvent 3.4 cut off, 4beedb8a26bed6c2ccf7ec...",technosphere
"'market for lubricating oil' (kilogram, GLO, None)",881,kilogram,"(ecoinvent 3.4 cut off, 73755d76796b5973edb546...",technosphere
"'market for copper' (kilogram, GLO, None)",335,kilogram,"(ecoinvent 3.4 cut off, d6f0a3a0933866c2f8d7eb...",technosphere
"'market for polystyrene foam slab' (kilogram, GLO, None)",1.6,kilogram,"(ecoinvent 3.4 cut off, f73ebbe3fe8fd9ac4f9f55...",technosphere
"'market for steel, low-alloyed, hot rolled' (kilogram, GLO, None)",1438,kilogram,"(ecoinvent 3.4 cut off, 1e200a3049db914305193e...",technosphere


### Comparision of inverters 2.5kW and 500kW

In [72]:
pd.concat([print_data2(inverter_2_5kW_adjusted),print_data2(inverter_500kW_adjusted)], axis = 1)

C:\Users\Scarlett Tannous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,unit,amount,unit
"'electricity, high voltage, production mix' (kilowatt hour, CN-GD, None)",0,kilowatt hour,0,kilowatt hour
"'electricity, high voltage, production mix' (kilowatt hour, FR, None)",0,kilowatt hour,0,kilowatt hour
"'inverter production, 2.5kW adjusted' (unit, RER, None)",1,unit,NaN,NaN
"'inverter production, 500kW adjusted' (unit, RER, None)",NaN,NaN,1,unit
"'market for alkyd paint, white, without solvent, in 60% solution state' (kilogram, GLO, None)",22,kilogram,22,kilogram
"'market for aluminium, cast alloy' (kilogram, GLO, None)",131,kilogram,131,kilogram
"'market for capacitor, electrolyte type, > 2cm height' (kilogram, GLO, None)",0.256,kilogram,0.256,kilogram
"'market for capacitor, film type, for through-hole mounting' (kilogram, GLO, None)",0.341,kilogram,0.341,kilogram
"'market for capacitor, tantalum-, for through-hole mounting' (kilogram, GLO, None)",0.023,kilogram,0.023,kilogram
"'market for copper' (kilogram, GLO, None)",335,kilogram,335,kilogram


In [91]:
def ensure_inverter_P_per_kg():
    
    inverter_500kW = [act for act in eidb if 'inverter production, 500kW' in act['name'] and not 'car' in act['name'] and 'RER' in act['location']][0]
    inverter_P_per_kg = [act for act in eidb if 'Inverter production, P kW per kg' in act['name']]
    
    if len(inverter_P_per_kg)==0:
        inverter_P_per_kg = inverter_500kW.copy()
        inverter_P_per_kg['name'] = 'Inverter production, P kW per kg'
        inverter_P_per_kg['unit'] = 'kilogram'
        inverter_P_per_kg['authors'] = 'Romain Besseau'
        inverter_P_per_kg['comment'] = 'Dataset based on the inventories of 3 kVA and 500 kVA PV inverters. It gives the inventory per inverter kilogram depending on the rated power P and considering a linear extrapolation between 3 kVA and 500 kVA inventories that do not have the same material ratio. The impact per kW will then be calculated considering 2 kg of inverter per kW (based on the analysis of ABB PV inverters). PS: With technical improvements the ratio weight power tends to decrease.'
        inverter_P_per_kg.save()
        
        
        #Removing all data
        for exc in inverter_P_per_kg.exchanges():
            exc.delete()
        inverter_P_per_kg.save()

        #adding the 2 inverters
        inverter_2_5kW_adjusted = [act for act in eidb if 'inverter production, 2.5kW adjusted' in act['name']][0]
        new_exc = inverter_P_per_kg.new_exchange(input = inverter_2_5kW_adjusted.key, 
                                                name = inverter_2_5kW_adjusted['name'], 
                                                amount = 1/18.5, 
                                                unit = inverter_2_5kW_adjusted['unit'],
                                                type = 'technosphere',
                                                #2*P represents the mass
                                                 formula = '(2*P)*((P-2.5)/(500-2.5))* (1/18.5)'
                                                )
        new_exc.save()
        
        inverter_500kW_adjusted = [act for act in eidb if 'inverter production, 500kW adjusted' in act['name']][0]
        new_exc = inverter_P_per_kg.new_exchange(input = inverter_500kW_adjusted.key, 
                                                   name = inverter_500kW_adjusted['name'], 
                                                   amount = 1/3000, unit = inverter_500kW_adjusted['unit'],
                                                   type = 'technosphere',
                                                 formula = '(2*P)*(1- ((P-2.5)/(500-2.5)))* (1/3000)'
                                                )

        new_exc.save()
        inverter_P_per_kg.save()
    
    
    else:
        inverter_P_per_kg = [act for act in eidb if 'Inverter production, P kW per kg' == act['name']][0]
        
    
    return inverter_P_per_kg

In [92]:
inverter_P_per_kg = ensure_inverter_P_per_kg()

In [ ]:
inverter_P_per_kg 

In [ ]:
ActivityParameter.update(amount=0.0).where(ActivityParameter.name == "eu_electricity_mix").execute()

In [96]:
inverter_P_per_kg = [act for act in eidb if 'Inverter production, P kW per kg' in act['name']][0]
inverter_P_per_kg

'Inverter production, P kW per kg' (kilogram, RER, None)

In [ ]:
value = np.interp(P, [3, 500], df_inverter_share.loc[exc][['3 kW share', '500 kW share']].values)

In [87]:
print_data(inverter_P_per_kg)

,amount,unit,key,type
"'inverter production, 2.5kW adjusted' (unit, RER, None)",0.0555556,unit,"(ecoinvent 3.4 cut off, 229b4d7410ec4980a674cd...",technosphere
"'inverter production, 500kW adjusted' (unit, RER, None)",0.000333333,unit,"(ecoinvent 3.4 cut off, 31392eff063b40f6b86f79...",technosphere


the idea is to have 2 uniform datasets where all the parameterized ds are set in the forumla
create 1 dataset including the 2.5 and the 500 and monitoring the amounts by the formla of the interpolation
the putting the created dataset in the final dictionary where the amount of inverter is controlled by the PV_lifetime and the inverter_lifetime

In [88]:
#To check with BG

## Metallization paste

### Original metallization ds

In [39]:
metal_paste = [act for act in eidb if 'metallization paste production, back side' == act['name'] and 'RER' in act['location'] and not 'aluminium' in act['name']][0]
metal_paste   

'metallization paste production, back side' (kilogram, RER, None)

### Modifying function metallization

In [94]:
#Create a new metalization paste copper based

def ensure_copper_metallization_paste():

    metal_paste_copper = [act for act in eidb if 'metallization paste production, back side, copper' == act['name'] and not 'aluminium' in act['name']]

    if len(metal_paste_copper) == 0: 
        metal_paste = [act for act in eidb if 'metallization paste production, back side' == act['name'] and 'RER' in act['location'] and not 'aluminium' in act['name']][0]
        metal_paste_copper = metal_paste.copy()
        metal_paste_copper['name'] = 'metallization paste production, back side, copper'
        metal_paste_copper['authors'] = 'Romain Besseau'
        metal_paste_copper['comment'] = 'Dataset for metallization paste production where Silver is simply replaced by Copper'

        for exc in metal_paste_copper.exchanges():
            exc['output'] = metal_paste_copper.key
            exc.save()

        copper_dataset = [act for act in eidb if 'copper production, primary' == act['name'] and 'RoW' in act['location']][0]
        #Changing the silver dataset to copper.               
        for exc in metal_paste_copper.exchanges():
            if exc['name'] == 'silver':
                exc['name'] = copper_dataset['name']
                exc['input'] = copper_dataset.key
                exc.save()  
        metal_paste_copper.save()  
    else: 
        metal_paste_copper = [act for act in eidb if 'metallization paste production, back side, copper' in act['name'] ][0]
    return metal_paste_copper

In [93]:
metal_paste_copper = ensure_copper_metallization_paste()

Activity to change:
silver
Activity changed:
copper production, primary


In [42]:
metal_paste_copper = [act for act in eidb if 'metallization paste production, back side, copper' in act['name']][0]
metal_paste_copper 

'metallization paste production, back side, copper' (kilogram, RER, None)

### Comparision of original and modified metallization paste

In [96]:
pd.concat([print_data2(metal_paste),print_data2(metal_paste_copper)], axis=1)

C:\Users\Scarlett Tannous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,unit,amount,unit
"'copper production, primary' (kilogram, RoW, None)",NaN,NaN,0.6767,kilogram
"'market for chemical, organic' (kilogram, GLO, None)",0.2525,kilogram,0.2525,kilogram
"'market for lead' (kilogram, GLO, None)",0.0808,kilogram,0.0808,kilogram
"'market for silver' (kilogram, GLO, None)",0.6767,kilogram,NaN,NaN
"'market for solder factory' (unit, GLO, None)",2e-10,unit,2e-10,unit
"'market group for electricity, medium voltage' (kilowatt hour, RER, None)",0.25,kilowatt hour,0.25,kilowatt hour
"'market group for heat, district or industrial, natural gas' (megajoule, RER, None)",0.7452,megajoule,0.7452,megajoule
"'metallization paste production, back side' (kilogram, RER, None)",1,kilogram,NaN,NaN
"'metallization paste production, back side, copper' (kilogram, RER, None)",NaN,NaN,1,kilogram


## Silicon

### Original silicon dataset

In [43]:
silicon_original = [act for act in eidb if 'silicon production, solar grade, modified Siemens process'  in act['name'] and 'RER' in act['location']][0]
print_data(silicon_original)

,amount,unit,key,type
"'market for hydrochloric acid, without water, in 30% solution state' (kilogram, RER, None)",1.6017,kilogram,"(ecoinvent 3.4 cut off, 0f10b7c564e830df89f7a2...",technosphere
"'market for sodium hydroxide, without water, in 50% solution state' (kilogram, GLO, None)",0.34819,kilogram,"(ecoinvent 3.4 cut off, 4530462f16af88054d371d...",technosphere
"'market group for heat, district or industrial, natural gas' (megajoule, RER, None)",185,megajoule,"(ecoinvent 3.4 cut off, c7dc74449ffe347834f24f...",technosphere
"'market for silicone factory' (unit, GLO, None)",1e-11,unit,"(ecoinvent 3.4 cut off, 2d3f757fac61524711ef90...",technosphere
"'electricity production, hydro, run-of-river' (kilowatt hour, RoW, None)",65,kilowatt hour,"(ecoinvent 3.4 cut off, 0ec1bfaa987050bf64a0e1...",technosphere
"'market for silicon, metallurgical grade' (kilogram, GLO, None)",1.13,kilogram,"(ecoinvent 3.4 cut off, 3188d23f89079fe1ac63e4...",technosphere
"'market group for electricity, high voltage' (kilowatt hour, RER, None)",45,kilowatt hour,"(ecoinvent 3.4 cut off, ad2c475320466cb2fd277e...",technosphere
"'market for hydrogen, liquid' (kilogram, RER, None)",0.050139,kilogram,"(ecoinvent 3.4 cut off, bf7545571e83c8827ce310...",technosphere
"'silicon production, solar grade, modified Siemens process' (kilogram, RER, None)",1,kilogram,"(ecoinvent 3.4 cut off, e3448d3e86354c368734d4...",production
"'COD, Chemical Oxygen Demand' (kilogram, None, ('water', 'surface water'))",0.00202,kilogram,"(biosphere3, fc0b5c85-3b49-42c2-a3fd-db7e57b69...",biosphere


### Parameterized silicon function

In [107]:
def ensure_silicon_adjusted():
    silicon_original = [act for act in eidb if 'silicon production, solar grade, modified Siemens process'  in act['name'] and 'RER' in act['location']][0]
    silicon_adjusted = [act for act in eidb if 'silicon production, solar grade, modified Siemens process adjusted' == act['name']]
    
    if len(silicon_adjusted) == 0:
        silicon_original = [act for act in eidb if 'silicon production, solar grade, modified Siemens process'  in act['name'] and 'RER' in act['location']][0]
        silicon_adjusted = silicon_original.copy()
        silicon_adjusted['name'] = 'silicon production, solar grade, modified Siemens process adjusted'
        silicon_adjusted['authors'] = 'Romain Besseau'
        silicon_adjusted['comment'] = 'Dataset adjusted with the amount of electricity and the corresponding dataset'      
        
        
        #Deleting hydro dataset not to count two times electricity
        # Elec dataset to remove to avoid double amount of electricity
        elec_hydro = [act for act in eidb if 'electricity production, hydro, run-of-river' in act['name'] and 'RoW' in act['location']][0]

        for exc in silicon_adjusted.exchanges():
            if exc['unit'] == 'kilowatt hour':
                if exc['activity'] == elec_hydro['activity']:
                    exc['amount'] = 0
                    exc.save()
        
        silicon_adjusted.save()
        
        ##################################
        for exc in silicon_adjusted.exchanges():
            if exc['name'] == 'electricity, high voltage':
                electricity_amount = exc
                exc['formula'] = 'silicon_electricity_intensity * default_electricity_mix'
                exc.save()
        silicon_adjusted.save() 
        
        for x in electricity_activity_w_location:
            act = x['activity']
            exc = inverter_500kW_adjusted.new_exchange(
                input = act.key,
                name="%s %s"%(x['name'], x['location']),
                amount=0.0,
                unit=electricity_amount['unit'],
                category='',
                type=electricity_amount['type'],
                formula='%f * %s'%(electricity_amount['amount'], x['varname']))
            exc.save()
        silicon_adjusted.save()
    else:
        #silicon = [act for act in eidb if initial_dataset in act['name'] and 'RER' in act['location']][0]
        silicon_adjusted = [act for act in eidb if initial_dataset + ' adjusted' in act['name']][0]
    
    return silicon_adjusted

In [108]:
silicon_adjusted = ensure_silicon_adjusted()

In [45]:
silicon_adjusted = [act for act in eidb if 'silicon production, solar grade, modified Siemens process adjusted' in act['name']][0]
print_data(silicon_adjusted)

,amount,unit,key,type
"'market for hydrochloric acid, without water, in 30% solution state' (kilogram, RER, None)",1.6017,kilogram,"(ecoinvent 3.4 cut off, 0f10b7c564e830df89f7a2...",technosphere
"'market for sodium hydroxide, without water, in 50% solution state' (kilogram, GLO, None)",0.34819,kilogram,"(ecoinvent 3.4 cut off, 4530462f16af88054d371d...",technosphere
"'market group for heat, district or industrial, natural gas' (megajoule, RER, None)",185,megajoule,"(ecoinvent 3.4 cut off, c7dc74449ffe347834f24f...",technosphere
"'market for silicone factory' (unit, GLO, None)",1e-11,unit,"(ecoinvent 3.4 cut off, 2d3f757fac61524711ef90...",technosphere
"'electricity production, hydro, run-of-river' (kilowatt hour, RoW, None)",0,kilowatt hour,"(ecoinvent 3.4 cut off, 0ec1bfaa987050bf64a0e1...",technosphere
"'market for silicon, metallurgical grade' (kilogram, GLO, None)",1.13,kilogram,"(ecoinvent 3.4 cut off, 3188d23f89079fe1ac63e4...",technosphere
"'market group for electricity, high voltage' (kilowatt hour, RER, None)",45,kilowatt hour,"(ecoinvent 3.4 cut off, ad2c475320466cb2fd277e...",technosphere
"'market for hydrogen, liquid' (kilogram, RER, None)",0.050139,kilogram,"(ecoinvent 3.4 cut off, bf7545571e83c8827ce310...",technosphere
"'silicon production, solar grade, modified Siemens process adjusted' (kilogram, RER, None)",1,kilogram,"(ecoinvent 3.4 cut off, 3d6705961f86481da10de9...",production
"'COD, Chemical Oxygen Demand' (kilogram, None, ('water', 'surface water'))",0.00202,kilogram,"(biosphere3, fc0b5c85-3b49-42c2-a3fd-db7e57b69...",biosphere


### Comparision of original and modified silicon ds

In [113]:
pd.concat([print_data2(silicon_original),print_data2(silicon_adjusted)], axis=1)

C:\Users\Scarlett Tannous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,amount,unit,amount,unit
"'AOX, Adsorbable Organic Halogen as Cl' (kilogram, None, ('water', 'surface water'))",1.2619e-05,kilogram,1.2619e-05,kilogram
"'BOD5, Biological Oxygen Demand' (kilogram, None, ('water', 'surface water'))",0.00020471,kilogram,0.00020471,kilogram
"'COD, Chemical Oxygen Demand' (kilogram, None, ('water', 'surface water'))",0.00202,kilogram,0.00202,kilogram
"'Chloride' (kilogram, None, ('water', 'surface water'))",0.035991,kilogram,0.035991,kilogram
"'Copper, ion' (kilogram, None, ('water', 'surface water'))",1.0236e-07,kilogram,1.0236e-07,kilogram
"'DOC, Dissolved Organic Carbon' (kilogram, None, ('water', 'surface water'))",0.00090998,kilogram,0.00090998,kilogram
"'Iron, ion' (kilogram, None, ('water', 'surface water'))",5.6085e-06,kilogram,5.6085e-06,kilogram
"'Nitrogen' (kilogram, None, ('water', 'surface water'))",0.00020751,kilogram,0.00020751,kilogram
"'Phosphate' (kilogram, None, ('water', 'surface water'))",2.8043e-06,kilogram,2.8043e-06,kilogram
"'Sodium, ion' (kilogram, None, ('water', 'surface water'))",0.03379,kilogram,0.03379,kilogram


## Wafer manufacturing

### Cutting process

#### Silicon carbide recycled

In [114]:
def generate_SiC_recycled_dataset():
    
    if [act for act in eidb if 'silicon carbide, recycled production' == act['name']] == []:
        act_original= [act for act in eidb if 'silicon carbide production' in act['name'] and 'RER' in act['location']][0]
        SiC_recycled=act_original.copy()
        SiC_recycled['name'] = 'silicon carbide, recycled production'
        SiC_recycled['unit'] = 'kilogram'
        SiC_recycled['author'] = 'ST based on KBOB:2016'
        SiC_recycled['location'] = 'RER'
        SiC_recycled['comment'] = 'Dataset corresponding to the Silicon carbide, recycling, at plant/kg/RER of the KBOB:2016'
        
        #Removing all data
        for exc in SiC_recycled.exchanges():
            exc.delete()
            exc.save()
        SiC_recycled.save()
        
        #Output
        act = [act for act in eidb if 'silicon carbide, recycled production' in act['name']][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = 1, unit = 'kilogram', type = 'production', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Electricity input
        act =[act for act in eidb if 'market group for electricity, medium voltage' in act['name'] and 'ENTSO' in act['location']][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = 0.78571, unit = 'kilowatt hour', type = 'technosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Transport input
        act = [act for act in eidb if 'market for transport, freight, lorry, unspecified' in act['name'] and 'GLO' in act['location']][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = 0.2625, unit = 'ton kilometer', type = 'technosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Silicon input
        act = [act for act in eidb if 'market for silicone factory' in act['name']][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = 1e-11, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Emissions to air
        act = [act for act in bio3 if 'Heat, waste' == act['name'] and "air" in act['categories'] and len(act['categories'])==1][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = 2.8286, unit = 'megajoule', type = 'biosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Disposal Si
        act = [act for act in eidb if 'market for waste, from silicon wafer production, inorganic' in act['name']][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = -0.042857, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Disposal antifreezer
        #This had double impact:
        #act = [act for act in eidb if 'market for spent antifreezer liquid' in act['name'] and 'GLO' in act['location']][0]
        #This should be used because it is not CH only
        #act = [act for act in eidb if 'treatment of spent antifreezer liquid, hazardous waste incineration' in act['name'] and 'RoW' in act['location']][0]
        act = [act for act in eidb if 'treatment of spent antifreezer liquid, hazardous waste incineration' in act['name'] and 'CH' in act['location']][0]
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = -0.071429, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        #Disposal pig iron
        act = [act for act in eidb if 'market for sludge, pig iron production' in act['name'] and 'Europe' in act['location']][0] 
        new_exc = SiC_recycled.new_exchange(input = act.key, amount = -0.13571, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        SiC_recycled.save()
        
        
    else:
        act_original= [act for act in eidb if 'silicon carbide production' in act['name'] and 'RER' in act['location']][0]
        SiC_recycled = [act for act in eidb if 'silicon carbide, recycled production' == act['name']][0]
                
    return SiC_recycled

In [115]:
SiC_recycled = generate_SiC_recycled_dataset()
print_data(SiC_recycled)

,amount,unit,key,type
"'silicon carbide, recycled production' (kilogram, RER, None)",1,kilogram,"(ecoinvent 3.4 cut off, b5dfc3c477814dfbaad6f1...",production
"'market group for electricity, medium voltage' (kilowatt hour, ENTSO-E, None)",0.78571,kilowatt hour,"(ecoinvent 3.4 cut off, cca4325e1d556f283ccd07...",technosphere
"'market for transport, freight, lorry, unspecified' (ton kilometer, GLO, None)",0.2625,ton kilometer,"(ecoinvent 3.4 cut off, 3200dcb1e1249ee6c024f9...",technosphere
"'market for silicone factory' (unit, GLO, None)",1e-11,unit,"(ecoinvent 3.4 cut off, 2d3f757fac61524711ef90...",technosphere
"'Heat, waste' (megajoule, None, ('air',))",2.8286,megajoule,"(biosphere3, f2d84834-d0b3-42e5-b41a-f04cc8033...",biosphere
"'market for waste, from silicon wafer production, inorganic' (kilogram, GLO, None)",-0.042857,kilogram,"(ecoinvent 3.4 cut off, 5bb57ef01519b55f255081...",technosphere
"'treatment of spent antifreezer liquid, hazardous waste incineration' (kilogram, CH, None)",-0.071429,kilogram,"(ecoinvent 3.4 cut off, 9765bbeefb54777d9aaa67...",technosphere
"'market for sludge, pig iron production' (kilogram, Europe without Switzerland, None)",-0.13571,kilogram,"(ecoinvent 3.4 cut off, 82616e4b3b11d10e667f77...",technosphere


In [46]:
SiC_recycled = [act for act in eidb if 'silicon carbide, recycled production' in act['name']][0]
SiC_recycled

'silicon carbide, recycled production' (kilogram, RER, None)

#### Ethylene glycol recycled 

In [116]:
def generate_TEG_recycled_dataset():
    
    if [act for act in eidb if 'triethylene glycol, recycled production' == act['name']] == []:
        act_original= [act for act in eidb if 'ethylene glycol production' in act['name'] and 'RER' in act['location']][0]
        TEG_recycled=act_original.copy()
        TEG_recycled['name'] = 'triethylene glycol, recycled production'
        TEG_recycled['unit'] = 'kilogram'
        TEG_recycled['author'] = 'ST based on KBOB:2016'
        TEG_recycled['location'] = 'RER'
        TEG_recycled['comment'] = 'Dataset corresponding to the Triethylene glycol, recycling, at plant/RER of the KBOB:2016'
        
        #Removing all data
        for exc in TEG_recycled.exchanges():
            exc.delete()
            exc.save()
        TEG_recycled.save()
        
        #Output
        act = [act for act in eidb if 'triethylene glycol, recycled production' in act['name']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = 1, unit = 'kilogram', type = 'production', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Electricity input
        act =[act for act in eidb if 'market group for electricity, medium voltage' in act['name'] and 'ENTSO' in act['location']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = 0.78571, unit = 'kilowatt hour', type = 'technosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Transport input
        act = [act for act in eidb if 'market for transport, freight, lorry, unspecified' in act['name'] and 'GLO' in act['location']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = 0.2625, unit = 'ton kilometer', type = 'technosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Silicon input
        act = [act for act in eidb if 'market for silicone factory' in act['name']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = 1e-11, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Emissions to air
        act = [act for act in bio3 if 'Heat, waste' in act['name'] and "air" in act['categories'] and 'f2d84834-d0b3-42e5-b41a-f04cc80337a4' in act['code']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = 2.8286, unit = 'megajoule', type = 'biosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Disposal Si
        act = act = [act for act in eidb if 'market for waste, from silicon wafer production, inorganic' in act['name']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = -0.042857, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Disposal antifreezer
        #This had double impact:
        #act = [act for act in eidb if 'market for spent antifreezer liquid' in act['name'] and 'GLO' in act['location']][0]
        #This should be used because it is not CH only
        #act = [act for act in eidb if 'treatment of spent antifreezer liquid, hazardous waste incineration' in act['name'] and 'RoW' in act['location']][0]
        act = [act for act in eidb if 'treatment of spent antifreezer liquid, hazardous waste incineration' in act['name'] and 'CH' in act['location']][0]
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = -0.071429, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        #Disposal pig iron
        act = [act for act in eidb if 'market for sludge, pig iron production' in act['name'] and 'Europe' in act['location']][0] 
        new_exc = TEG_recycled.new_exchange(input = act.key, amount = -0.13571, unit = 'unit', type = 'technosphere', name = act['name'])
        new_exc.save()
        TEG_recycled.save()
        
        
    else:
        act_original= [act for act in eidb if 'ethylene glycol production' in act['name'] and 'RER' in act['location']][0]
        TEG_recycled = [act for act in eidb if 'triethylene glycol, recycled production' == act['name']][0]
                
    return TEG_recycled

In [117]:
TEG_recycled = generate_TEG_recycled_dataset()
TEG_recycled

'triethylene glycol, recycled production' (kilogram, RER, None)

In [47]:
TEG_recycled = [act for act in eidb if 'triethylene glycol, recycled production' in act['name']][0] 
TEG_recycled

'triethylene glycol, recycled production' (kilogram, RER, None)

#### Diamond wiring

In [118]:
def generate_tungsten():
    '''
    This function generates a new dataset the tungsten carbide (expressed in kg) which is used for the wire later on.
    '''
    
    if [act for act in eidb if 'tungsten carbide production' == act['name']] == []:
        act_original=[act for act in eidb if 'silicon carbide production' == act['name'] and 'RER' in act['location']][0]
        tungsten_carbide=act_original.copy()
        tungsten_carbide['name'] = 'tungsten carbide production'
        tungsten_carbide['unit'] = 'kilogram'
        tungsten_carbide['location'] = 'RER'
        tungsten_carbide['author'] = 'ST based on Isabella Bianco dataset'
        tungsten_carbide['comment'] = 'Dataset corresponding to the LCI provided by Isabella Bianco(2018) thesis - Table 46'
        
        #Removing all data
        for exc in tungsten_carbide.exchanges():
            exc.delete()
            exc.save()
        tungsten_carbide.save()
        
        #Output
        act = [act for act in eidb if 'tungsten carbide production' in act['name']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 1, unit = 'kilogram', type = 'production', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        #input
        act = [act for act in eidb if 'market for sodium hydroxide, without water, in 50% solution state' in act['name'] and 'GLO' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 1.49, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'aluminium sulfate production, powder' in act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.071, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'sodium sulfide production' in act['name'] and 'GLO' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.044, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'magnesium sulfate production' in act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.027, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'market for ammonia, liquid' in act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.106, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'market for soda ash, dense' in act['name'] and 'GLO' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 1.21, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'sulfuric acid production' == act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 1.24, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'market group for electricity, medium voltage' in act['name'] and 'ENTSO-E' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 111.16, unit = 'kilowatt hour', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'hydrogen sulfide production' == act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.0076, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'market for nitrogen, liquid' == act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 1.65, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'market for hydrogen, liquid' in act['name'] and 'RER' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.32, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()        
        
        act = [act for act in eidb if 'market for tap water' in act['name'] and 'RoW' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 2.84, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
        act = [act for act in eidb if 'carbon black production' == act['name'] and 'GLO' in act['location']][0]
        new_exc = tungsten_carbide.new_exchange(input = act.key, amount = 0.13, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        tungsten_carbide.save()
        
    else:
        tungsten_carbide = [act for act in eidb if 'tungsten carbide production' == act['name']][0]
        
    return tungsten_carbide  

In [119]:
tungsten_carbide = generate_tungsten()

'tungsten carbide production' (kilogram, RER, None)

In [49]:
tungsten_carbide = [act for act in eidb if 'tungsten carbide production' in act['name']][0]
tungsten_carbide

'tungsten carbide production' (kilogram, RER, None)

In [120]:
def generate_diamond_powder():
    '''
    This function generates a new dataset of diamond powder which is used for the wire later on.
    '''
    
    if [act for act in eidb if 'diamond powder, production' == act['name']] == []:
        act_original= [act for act in eidb if 'sodium percarbonate production, powder' in act['name'] and 'RER' in act['location']][0]
        diamond_powder=act_original.copy()
        diamond_powder['name'] = 'diamond powder, production'
        diamond_powder['unit'] = 'kilogram'
        diamond_powder['location'] = 'RER'
        diamond_powder['author'] = 'ST based on Isabella Bianco dataset'
        diamond_powder['comment'] = 'Dataset corresponding to the LCI provided by Isabella Bianco(2018) thesis - Table 51'
        
        #Removing all data
        for exc in diamond_powder.exchanges():
            exc.delete()
            exc.save()
        diamond_powder.save()
        
        #Output
        act = [act for act in eidb if 'diamond powder, production' in act['name']][0]
        new_exc = diamond_powder.new_exchange(input = act.key, amount = 1, unit = 'kilogram', type = 'production', name = act['name'])
        new_exc.save()
        diamond_powder.save()
        
        #input
        act = [act for act in eidb if 'market for electricity, medium voltage' in act['name'] and 'CN-CSG' in act['location']][0]
        new_exc = diamond_powder.new_exchange(input = act.key, amount = 840, unit = 'kilowatt hour', type = 'technosphere', name = act['name'])
        new_exc.save()
        diamond_powder.save()
        
        act = [act for act in eidb if 'graphite production' in act['name'] and 'RER' in act['location']][0]
        new_exc = diamond_powder.new_exchange(input = act.key, amount = 1, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        diamond_powder.save()
        
        #transport from China to Europe (Italy, for example)
        act = [act for act in eidb if 'transport, freight, sea, transoceanic ship' == act['name'] and 'GLO' in act['location']][0]
        new_exc = diamond_powder.new_exchange(input = act.key, amount = 170, unit = 'ton kilometer', type = 'technosphere', name = act['name'])
        new_exc.save()
        diamond_powder.save()
        
        act = [act for act in eidb if 'transport, freight, lorry 16-32 metric ton, EURO4' == act['name'] and 'RER' in act['location']][0]
        new_exc = diamond_powder.new_exchange(input = act.key, amount = 0.53, unit = 'ton kilometer', type = 'technosphere', name = act['name'])
        new_exc.save()
        diamond_powder.save()
        
    else:
        diamond_powder = [act for act in eidb if 'diamond powder, production' == act['name']][0]
        
    return diamond_powder

In [121]:
diamond_powder = generate_diamond_powder()

'diamond powder, production' (kilogram, RER, None)

In [51]:
diamond_powder = [act for act in eidb if  'diamond powder, production' in act['name']][0]
diamond_powder 

'diamond powder, production' (kilogram, RER, None)

In [122]:
def generate_sintered_diamond_bead():
    
    '''
    This function generates a new dataset of sintered diamond which is used for the wire later on.
    The paramater elec_dataset permits the control of the electricity used for manufacturing.
    '''
    
    if [act for act in eidb if 'sintered diamond bead, for quarrying' == act['name']] == []:
        act_original= [act for act in eidb if  'polymethyl methacrylate production, beads' == act['name'] and 'RER' in act['location']][0]
        sintered_diamond_bead=act_original.copy()
        sintered_diamond_bead['name'] = 'sintered diamond bead, for quarrying'
        sintered_diamond_bead['unit'] = 'kilogram'
        sintered_diamond_bead['location'] = 'RER'
        sintered_diamond_bead['author'] = 'ST based on Isabella Bianco dataset'
        sintered_diamond_bead['comment'] = 'Dataset corresponding to the LCI provided by Isabella Bianco(2018) thesis - Table 50'
        
        #Removing all data
        for exc in sintered_diamond_bead.exchanges():
            exc.delete()
            exc.save()
        sintered_diamond_bead.save()
        
        #Output
        act = [act for act in eidb if 'sintered diamond bead, for quarrying' in act['name']][0]
        new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 0.004, unit = 'kilogram', type = 'production', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
    
        #the input of the sintered diamond bead (table 50 of Isabella Bianco thesis) to be integrated to Table 45
        
        act = [act for act in eidb if 'market group for electricity, medium voltage' == act['name'] and 'ENTSO-E' == act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 0.1, unit = 'kilowatt hour', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'market for steel, unalloyed' in act['name'] and 'GLO' in act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 0.003, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'market for cobalt' in act['name'] and 'GLO' in act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 7e-08, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()    
        
        act = [act for act in eidb if 'market for copper' == act['name']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 2.1e-07, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if  'market for pig iron' == act['name']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 3.5e-07, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'market for nickel, 99.5%' == act['name']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 2.1e-08, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'tungsten carbide production' == act['name'] and 'RER' in act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 3.5e-08, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'diamond powder, production' == act['name']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 2e-05, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if  'polymethyl methacrylate production, beads' == act['name'] and 'RER' in act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 2e-05, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'cobalt production' == act['name']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 5e-05, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'market for silver' in act['name']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 2.1e-04, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'market for nitrogen, liquid' in act['name'] and 'RER' in act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 0.0323, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
        
        act = [act for act in eidb if 'market for hydrogen, liquid' in act['name'] and 'RER' in act['location']][0]
        new_exc = new_exc = sintered_diamond_bead.new_exchange(input = act.key, amount = 9e-04, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_diamond_bead.save()
    
    else:
        sintered_diamond_bead = [act for act in eidb if 'sintered diamond bead, for quarrying' == act['name']][0]
                
    return sintered_diamond_bead 

In [123]:
sintered_diamond_bead = generate_sintered_diamond_bead()
sintered_diamond_bead = [act for act in eidb if 'sintered diamond bead, for quarrying' in act['name']][0]

'sintered diamond bead, for quarrying' (kilogram, RER, None)

In [52]:
sintered_diamond_bead = [act for act in eidb if 'sintered diamond bead, for quarrying' in act['name']][0]
sintered_diamond_bead 

'sintered diamond bead, for quarrying' (kilogram, RER, None)

In [124]:
def generate_DW_single_electroplated():
    
    '''
    This function generates a new dataset represented the electroplated diamond wire used for the cutting process of the wafer.
    The paramater elec_dataset permits the control of the electricity used for manufacturing.
    '''
    
    if [act for act in eidb if 'sintered diamond wire, steel' == act['name']] == []:
        act_original= [act for act in eidb if 'wire drawing, steel' in act['name'] and 'RER' in act['location']][0]
        sintered_DW=act_original.copy()
        sintered_DW['name'] = 'sintered diamond wire, steel'
        sintered_DW['unit'] = 'meter'
        sintered_DW['location'] = 'RER'
        sintered_DW['author'] = 'ST based on Isabella Bianco dataset'
        sintered_DW['comment'] = 'Dataset corresponding to the LCI provided by Isabella Bianco(2018) thesis - Table 47'
        
        #Removing all data
        for exc in sintered_DW.exchanges():
            exc.delete()
            exc.save()
        sintered_DW.save()
        
        #Output
        act = [act for act in eidb if 'sintered diamond wire, steel' in act['name']][0]
        new_exc = sintered_DW.new_exchange(input = act.key, amount = 1, unit = 'meter', type = 'production', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        #the input of the sintered diamond wire for primapry cuttings (Table 47)
        act = [act for act in eidb if 'sintered diamond bead, for quarrying' in act['name']][0]
        new_exc =  sintered_DW.new_exchange(input = act.key, amount = 0.0427, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        act = [act for act in eidb if 'market group for electricity, medium voltage' == act['name'] and 'ENTSO-E' == act['location']][0]
        new_exc = sintered_DW.new_exchange(input = act.key, amount = 0.9, unit = 'kilowatt hour', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        act = [act for act in eidb if 'wire drawing, steel' == act['name'] and 'RER' in act['location']][0]
        new_exc =  sintered_DW.new_exchange(input = act.key, amount = 0.095, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        act = [act for act in eidb if 'polyurethane production, flexible foam' in act['name'] and 'RER' in act['location']][0]
        new_exc =  sintered_DW.new_exchange(input = act.key, amount = 0.15, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        act = [act for act in eidb if 'transport, freight, lorry 16-32 metric ton, EURO4' == act['name'] and 'RER' in act['location']][0]
        new_exc =  sintered_DW.new_exchange(input = act.key, amount = 0.003, unit = 'ton kilometer', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        act = [act for act in eidb if 'market for waste plastic, mixture' == act['name'] and 'Europe' in act['location']][0]
        new_exc =  sintered_DW.new_exchange(input = act.key, amount = 0.045, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        sintered_DW.save()
        
        
    else:
        sintered_DW = [act for act in eidb if 'sintered diamond wire, steel' == act['name']][0]
    
    return sintered_DW

In [125]:
sintered_DW = generate_DW_single_electroplated()

'sintered diamond wire, steel' (meter, RER, None)

In [53]:
sintered_DW = [act for act in eidb if 'sintered diamond wire, steel' in act['name']][0]
sintered_DW

'sintered diamond wire, steel' (meter, RER, None)

### Wafer itself

In [60]:
act = [act for act in eidb if 'silicon production, solar grade, modified Siemens process adjusted' in act['name'] and 'RER' in act['location']][0]
act

'silicon production, solar grade, modified Siemens process adjusted' (kilogram, RER, None)

In [61]:
wafer_original = [act for act in eidb
                      if 'multi-Si wafer production' == act['name'] 
                      and 'RER' in act['location']][0]
print_data(wafer_original)

,amount,unit,key,type
"'market for sodium hydroxide, without water, in 50% solution state' (kilogram, GLO, None)",0.015,kilogram,"(ecoinvent 3.4 cut off, 4530462f16af88054d371d...",technosphere
"'market for triethylene glycol' (kilogram, GLO, None)",2.71,kilogram,"(ecoinvent 3.4 cut off, a15faee7df418629ee71a6...",technosphere
"'market for glass wool mat' (kilogram, GLO, None)",0.01,kilogram,"(ecoinvent 3.4 cut off, 915d0ca8f01d0edb98ef75...",technosphere
"'market for silicon carbide' (kilogram, GLO, None)",2.63,kilogram,"(ecoinvent 3.4 cut off, 072b42124a1c371931d3b5...",technosphere
"'market for waste, from silicon wafer production' (kilogram, GLO, None)",-0.17,kilogram,"(ecoinvent 3.4 cut off, 53500430763bc604083c36...",technosphere
"'market for dipropylene glycol monomethyl ether' (kilogram, GLO, None)",0.3,kilogram,"(ecoinvent 3.4 cut off, 727d8684a6083e413f9aa8...",technosphere
"'market for alkylbenzene sulfonate, linear, petrochemical' (kilogram, GLO, None)",0.24,kilogram,"(ecoinvent 3.4 cut off, d667de99da77e65830bb3d...",technosphere
"'market for packaging film, low density polyethylene' (kilogram, GLO, None)",0.1,kilogram,"(ecoinvent 3.4 cut off, 9bf8604690f4fad29b9981...",technosphere
"'market for acrylic binder, without water, in 34% solution state' (kilogram, GLO, None)",0.002,kilogram,"(ecoinvent 3.4 cut off, 64dd86ca92b77ef8f03eb2...",technosphere
"'market for water, completely softened, from decarbonised water, at user' (kilogram, GLO, None)",65,kilogram,"(ecoinvent 3.4 cut off, 132d9f6734f4ee99690b6f...",technosphere


In [62]:
act = [act for act in eidb if 'silicon production, solar grade, modified Siemens process adjusted' in act['name']][0]
act

['silicon production, solar grade, modified Siemens process adjusted' (kilogram, RER, None)]

In [77]:
def ensure_wafer_dataset():
    
    
    wafer_original = [act for act in eidb
                      if 'multi-Si wafer production' == act['name'] 
                      and 'RER' in act['location']][0]
    wafer_adjusted = [act for act in eidb if 'multi-Si wafer production adjusted' == act['name']]
    
    if len(wafer_adjusted) == 0: 
        wafer_original = [act for act in eidb
                      if 'multi-Si wafer production' == act['name'] 
                      and 'RER' in act['location']][0]
        wafer_adjusted = wafer_original.copy()
        wafer_adjusted['name'] = 'multi-Si wafer production adjusted'
        wafer_adjusted['authors'] = 'Romain Besseau and Scarlett Tannous'
        wafer_adjusted['comment'] = 'Dataset adjusted with the adjustment of the amount of electricity, the corresponding dataset, the amount of Silicon and the corresponding Silicon solar grade dataset'      
        wafer_adjusted.save()
        
        
        
        silicon_adjusted = [act for act in eidb if
                            'silicon production, solar grade, modified Siemens process adjusted' 
                            in act['name'] and 'RER' in act['location']][0]
        new_exc = silicon_adjusted.new_exchange(input = silicon_adjusted.key,
                                                amount = 0.0, unit = 'kilogram',
                                                type = 'technosphere', 
                                                name = silicon_adjusted['name'])
        new_exc.save()
        silicon_adjusted.save()
                
               
        act = [act for act in eidb if 'sintered diamond wire, steel' in act['name'] and 'RER' in act['location']][0]
        new_exc = wafer_adjusted.new_exchange(input = act.key, amount = 0, unit = 'meter', type = 'technosphere', name = act['name'])
        new_exc.save()
        wafer_adjusted.save()
        
        act = [act for act in eidb if 'silicon carbide, recycled production' in act['name']][0]
        new_exc = wafer_adjusted.new_exchange(input = act.key, amount = 0, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        
        #Adjusting TEG recycled
        act = [act for act in eidb if 'triethylene glycol, recycled production' in act['name'] and 'RER' in act['location']][0]
        new_exc = wafer_adjusted.new_exchange(input = act.key, amount = 0, unit = 'kilogram', type = 'technosphere', name = act['name'])
        new_exc.save()
        
        #Adding the 4 MJ natural gas used for the wafer as proposed by KBOB
        act = [act for act in eidb if 'at industrial furnace >100kW' in act['name'] and 'Europe' in act['location']][0]
        new_exc = wafer_adjusted.new_exchange(input = act.key, amount = 4, unit = 'megajoule', type = 'technosphere', name = act['name'])
        new_exc.save()
        
        #adding the adjusted silicon
        act = [act for act in eidb if 'silicon production, solar grade, modified Siemens process adjusted' in act['name']][0]
        new_exc = wafer_adjusted.new_exchange(input = act.key, amount = 4, unit = 'megajoule', type = 'technosphere', name = act['name'])
        new_exc.save()
        
        #adding the adjusted silicon
        act = [act for act in eidb if 'silicon production, solar grade, modified Siemens process adjusted' in act['name']][0]
        new_exc = wafer_adjusted.new_exchange(input = act.key, amount = 0, unit = 'megajoule', type = 'technosphere', name = act['name'])
        new_exc.save()
        
        
        for exc in wafer_adjusted.exchanges():
            if exc['name'] == 'silicon, multi-Si, casted':
                silicon_amount = exc['amount']
                exc['amount'] = 0
                exc.save()
            if exc['name'] == 'silicon production, solar grade, modified Siemens process adjusted':
                exc['amount'] = silicon_amount
                exc.save()
                
        wafer_adjusted.save()  

            

    else:
        #wafer = [act for act in eidb if initial_dataset == act['name'] and 'RER' in act['location']][0]
        wafer_adjusted = [act for act in eidb if 'multi-Si wafer production adjusted' in act['name']][0]
        print(
            "By selecting the diamond wiring process: " 
            +"it is advised to consider 15% less silicon used (translated into 15% less kerf loss) "
            +"when compared to the traditional cutting process (44%)"
        )   
    #if DW false: DW = 1, if true DW = 0
    DW = 1
    for exc in wafer_adjusted.exchanges():
                    
            if exc['name'] == 'silicon carbide' :
                SiC_amount = exc['name']
                SiC_amount_adjusted = 2.02988
                exc['formula'] = 'DW*%f*(1.0 - SiC_recycled_share)'%(SiC_amount_adjusted,)
                #exc['amount'] = 0
                exc.save()
                
            if exc['name'] == 'triethylene glycol' :
                TEG_amount = exc['name']
                TEG_amount_adjusted = 2.16548
                exc['formula'] = 'DW*%f* (1.0 - TEG_recycled_share)'%(TEG_amount_adjusted,)
                #exc['amount'] = 0
                exc.save()
                
            if exc['name'] == 'silicon carbide, recycled production':
                exc['formula'] = 'DW * SiC_amount_adjusted* SiC_recycled_share'
                exc.save()
            
            if exc['name'] == 'triethylene glycol, recycled production':
                exc['formula'] = 'DW * TEG_amount_adjusted* TEG_recycled_share'
                exc.save()
       
            if exc['name'] == 'tap water' :
                tapwater_amount = exc['amount']
                exc['formula'] = '%f + (1.0-DW)*98.94'%(tapwater_amount)
                #exc['amount'] = exc['amount'] + 98.94
                exc.save()
                
            if exc['name'] == 'sintered diamond wire, steel' :
                exc['formula'] = '(1.0-DW)*0.0006075'
                #exc['amount'] = 0.0006075
                exc.save()

            if exc['name'] == 'electricity, medium voltage':
                exc['amount'] = 20  # It was 8 but it was modified by ST to 20
                elec_amount_wafer = exc['amount']
                exc['formula']= 'elec_amount_wafer * (1.0 - manufacturing_efficiency_gains)'
                exc.save()
            
            #Adjusting the amount of solar grade silicon used
            #Amount calculated from wafer thickness and Silicon density (2328 kg/m3)
            if exc['name'] == 'silicon production, solar grade, modified Siemens process adjusted':
                exc['formula'] = 'wafer_thickness*1e-06*2328 / (1-kerf_loss)'
                exc.save()
            #AskBenoit
            
            #Adjusting the electricity used for manufacturing
            if exc['name'] == 'electricity, medium voltage':
                    electricity_amount = exc
                    exc['formula'] = '%f * default_electricity_mix'%(electricity_amount['amount'])
                    exc.save()
                    
            wafer_adjusted.save()
        
    
    for x in electricity_activity_w_location:
        act = x['activity']
        exc = wafer_adjusted.new_exchange(
            input = act.key,
            name="%s %s"%(x['name'], x['location']),
            amount=0.0,
            unit=electricity_amount['unit'],
            category='',
            type=electricity_amount['type'],
            formula='%f * %s'%(electricity_amount['amount'], x['varname']))
        exc.save()
         
    wafer_adjusted.save()
    
    return wafer_adjusted

In [78]:
wafer_adjusted = ensure_wafer_dataset()
wafer_adjusted

By selecting the diamond wiring process: it is advised to consider 15% less silicon used (translated into 15% less kerf loss) when compared to the traditional cutting process (44%)


'multi-Si wafer production adjusted' (square meter, RER, None)

In [79]:
print_data(wafer_adjusted)

,amount,unit,key,type
"'market for sodium hydroxide, without water, in 50% solution state' (kilogram, GLO, None)",0.015,kilogram,"(ecoinvent 3.4 cut off, 4530462f16af88054d371d...",technosphere
"'market for triethylene glycol' (kilogram, GLO, None)",2.71,kilogram,"(ecoinvent 3.4 cut off, a15faee7df418629ee71a6...",technosphere
"'market for glass wool mat' (kilogram, GLO, None)",0.01,kilogram,"(ecoinvent 3.4 cut off, 915d0ca8f01d0edb98ef75...",technosphere
"'market for silicon carbide' (kilogram, GLO, None)",2.63,kilogram,"(ecoinvent 3.4 cut off, 072b42124a1c371931d3b5...",technosphere
"'market for waste, from silicon wafer production' (kilogram, GLO, None)",-0.17,kilogram,"(ecoinvent 3.4 cut off, 53500430763bc604083c36...",technosphere
"'market for dipropylene glycol monomethyl ether' (kilogram, GLO, None)",0.3,kilogram,"(ecoinvent 3.4 cut off, 727d8684a6083e413f9aa8...",technosphere
"'market for alkylbenzene sulfonate, linear, petrochemical' (kilogram, GLO, None)",0.24,kilogram,"(ecoinvent 3.4 cut off, d667de99da77e65830bb3d...",technosphere
"'market for packaging film, low density polyethylene' (kilogram, GLO, None)",0.1,kilogram,"(ecoinvent 3.4 cut off, 9bf8604690f4fad29b9981...",technosphere
"'market for acrylic binder, without water, in 34% solution state' (kilogram, GLO, None)",0.002,kilogram,"(ecoinvent 3.4 cut off, 64dd86ca92b77ef8f03eb2...",technosphere
"'market for water, completely softened, from decarbonised water, at user' (kilogram, GLO, None)",65,kilogram,"(ecoinvent 3.4 cut off, 132d9f6734f4ee99690b6f...",technosphere


In [84]:
for exc in wafer_adjusted.exchanges():
    display(exc.as_dict())

{'flow': '61396bcb-bf35-411a-a6a6-8543ccef83e8',
 'type': 'technosphere',
 'name': 'sodium hydroxide, without water, in 50% solution state',
 'classifications': {'CPC': ['34231: Chemical elements n.e.c.; inorganic acids except phosphoric, nitric and sulphonitric; inorganic oxygen compoun[…]']},
 'production volume': 0.0,
 'activity': '6ecb40f2-fefe-42d8-8e28-1c1e926b5c03',
 'unit': 'kilogram',
 'comment': 'De Wild 2007, for wafer cleaning',
 'amount': 0.015,
 'pedigree': {'reliability': 1,
  'completeness': 2,
  'temporal correlation': 4,
  'geographical correlation': 1,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': -4.199705077879927,
 'scale': 0.09,
 'scale without pedigree': 0.0,
 'input': ('ecoinvent 3.4 cut off', '4530462f16af88054d371de2dbe73947'),
 'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c')}

{'flow': 'e53a275d-a2c3-40d4-8dc0-df01de6068c9',
 'type': 'technosphere',
 'name': 'triethylene glycol',
 'classifications': {'CPC': ['34170: Ethers, alcohol peroxides, ether peroxides, epoxides, acetals and hemiacetals, and their halogenated, sulphona[…]']},
 'production volume': 0.0,
 'activity': 'b2293b47-09a2-4057-9dcb-8c39d9e721d6',
 'unit': 'kilogram',
 'comment': 'EcoSpold01Location=RER',
 'amount': 2.71,
 'pedigree': {'reliability': 1,
  'completeness': 2,
  'temporal correlation': 4,
  'geographical correlation': 1,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': 0.9969486348916096,
 'scale': 0.09,
 'scale without pedigree': 0.0,
 'input': ('ecoinvent 3.4 cut off', 'a15faee7df418629ee71a6ae45bd1e60'),
 'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'formula': 'DW*2.165480* (1.0 - TEG_recycled_share)'}

{'flow': 'a6ec2991-f0f9-42fe-b71c-995e50838b9b',
 'type': 'technosphere',
 'name': 'glass wool mat',
 'classifications': {'CPC': ['37116: Glass mirrors; multiple walled insulating units of glass']},
 'production volume': 0.0,
 'activity': '3f5d24d6-13a2-4a30-a695-afdb0219d21c',
 'unit': 'kilogram',
 'comment': 'De Wild 2007, for temporarily attachment of bricks to wire sawing equipment',
 'amount': 0.01,
 'pedigree': {'reliability': 2,
  'completeness': 2,
  'temporal correlation': 4,
  'geographical correlation': 1,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': -4.605170185988091,
 'scale': 0.09327379053088815,
 'scale without pedigree': 0.0,
 'input': ('ecoinvent 3.4 cut off', '915d0ca8f01d0edb98ef75f4012001fc'),
 'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c')}

{'flow': '626d96ee-f1a6-444d-9b67-f8e6f8e29d22',
 'type': 'technosphere',
 'name': 'silicon carbide',
 'classifications': {'CPC': ['373: Refractory products and structural non-refractory clay products']},
 'production volume': 0.0,
 'activity': '0ceff643-e765-458a-acf2-e732aeecfb51',
 'unit': 'kilogram',
 'comment': 'De Wild 2007, SiC use for sawing',
 'amount': 2.63,
 'pedigree': {'reliability': 1,
  'completeness': 2,
  'temporal correlation': 4,
  'geographical correlation': 1,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': 0.9669838461896731,
 'scale': 0.09,
 'scale without pedigree': 0.0,
 'input': ('ecoinvent 3.4 cut off', '072b42124a1c371931d3b51f56cf89cf'),
 'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'formula': 'DW*2.029880*(1.0 - SiC_recycled_share)'}

{'flow': '01246895-e90d-43e4-9e35-ff4031c458f7',
 'type': 'technosphere',
 'name': 'waste, from silicon wafer production',
 'classifications': {'CPC': ['39990: Other wastes n.e.c.']},
 'production volume': 442000.0,
 'activity': '2e19b0dd-5eb8-44b7-b11c-3042a051d2e7',
 'unit': 'kilogram',
 'comment': 'EcoSpold01Location=DE',
 'amount': -0.17,
 'pedigree': {'reliability': 1,
  'completeness': 2,
  'temporal correlation': 4,
  'geographical correlation': 1,
  'further technological correlation': 1},
 'uncertainty type': 2,
 'loc': -1.7719568419318752,
 'scale': 0.09,
 'scale without pedigree': 0.0,
 'input': ('ecoinvent 3.4 cut off', '53500430763bc604083c3643bbad4321'),
 'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c')}

...

{'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'input': ('ecoinvent 3.4 cut off', 'cca4325e1d556f283ccd07486880b832'),
 'name': 'market group for electricity, medium voltage ENTSO-E',
 'amount': 0.0,
 'unit': 'kilowatt hour',
 'category': '',
 'type': 'technosphere',
 'formula': '20.000000 * eu_electricity_mix'}

{'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'input': ('ecoinvent 3.4 cut off', '6e74ccf143f1ec23223bd8287ba00b3b'),
 'name': 'electricity, high voltage, production mix CN-GD',
 'amount': 0.0,
 'unit': 'kilowatt hour',
 'category': '',
 'type': 'technosphere',
 'formula': '20.000000 * cn_electricity_mix'}

{'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'input': ('ecoinvent 3.4 cut off', '737173be669aae5c03ed06230c59f808'),
 'name': 'electricity, high voltage, production mix FR',
 'amount': 0.0,
 'unit': 'kilowatt hour',
 'category': '',
 'type': 'technosphere',
 'formula': '20.000000 * fr_electricity_mix'}

{'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'input': ('ecoinvent 3.4 cut off', 'c5c72d58126d651e236b3eaeebe1c40e'),
 'name': 'market for electricity, high voltage US-WECC',
 'amount': 0.0,
 'unit': 'kilowatt hour',
 'category': '',
 'type': 'technosphere',
 'formula': '20.000000 * us_electricity_mix'}

{'output': ('ecoinvent 3.4 cut off', '8bbbcc85104143a289a95692768e582c'),
 'input': ('ecoinvent 3.4 cut off', 'f19e0fadcb5b20607b63798a44720d4a'),
 'name': 'market for electricity, high voltage RoW',
 'amount': 0.0,
 'unit': 'kilowatt hour',
 'category': '',
 'type': 'technosphere',
 'formula': '20.000000 * row_electricity_mix'}